In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Importazione delle librerie (request e BeautifulSoup servono per fare scraping)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [14]:

# ===================
# download pagina web
# ===================
wiki_url = "https://it.wikipedia.org/wiki/Demografia_d%27Italia"
pagina_wikipedia = requests.get(wiki_url).text


# ==============================
# creazione BeautifulSoup object
# per il parsing di pagine html
# ==============================
soup = BeautifulSoup(pagina_wikipedia, 'html.parser')


# ==========================================================
# recupero delle sezioni della pagina web con il tag "tabel"
# ==========================================================
tabelle = soup.find_all('table')
# for t in tabelle:
#     print(t.get('class'))




# ==============================
# creazione di un oggetto pandas
# ==============================
data_list = pd.read_html(str(tabelle[1]))
df = pd.DataFrame(data_list[0])
display(df)

Anno  Popolazione 1º gennaio          Natalità  \
                 Anno  Popolazione 1º gennaio              Nati   
0                1862              26.328.000           991.000   
1                1863              26.507.000         1.026.000   
2                1864              26.712.000         1.000.000   
3                1865              26.915.000         1.024.000   
4                1866              27.131.000         1.044.000   
..                ...                     ...               ...   
160          2022[22]              59.030.133           392.598   
161          2023[22]              58.850.717                 -   
162              Anno  Popolazione 1º gennaio          Natalità   
163              Anno  Popolazione 1º gennaio              Nati   
164  * = dati stimati        * = dati stimati  * = dati stimati   

                              Mortalità                      Saldo naturale  \
            Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
0                 375           815.000               309          +176.000   
1                 386           824.000               310          +202.000   
2                 373           802.000               299          +198.000   
3                 379           812.000               300          +212.000   
4                 383           798.000               293          +246.000   
..                ...               ...               ...               ...   
160                67           713.499               121          -320.901   
161                 -                 -                 -                 -   
162          Natalità         Mortalità         Mortalità    Saldo naturale   
163         Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
164  * = dati stimati  * = dati stimati  * = dati stimati  * = dati stimati   

    Tasso di fecondità totale[10][21]  
    Tasso di fecondità totale[10][21]  
0                                 493  
1                                 490  
2                                 490  
3                                 491  
4                                 491  
..                                ...  
160                               124  
161                               NaN  
162         Tasso di fecondità totale  
163         Tasso di fecondità totale  
164                  * = dati stimati  

[165 rows x 8 columns]

In [15]:
# ====================================
# estrazione dati in forma di stringhe
# ====================================
anno_str        = df['Anno']['Anno'].to_numpy()[:-4]
popolazione_str = df["Popolazione 1º gennaio"]["Popolazione 1º gennaio"].to_numpy()[:-4]
nati_str        = df["Natalità"]["Nati"].to_numpy()[:-4]
morti_str       = df["Mortalità"]["Morti"].to_numpy()[:-4]


In [17]:
# =================================================================
# In questo caso le stringhe che descrivono i valori usano il punto
# come separatore per le migliaia ed è necessario elimiarlo
# prima di convertire in interi
# =================================================================
def conversione_stringhe_interi(data_str: np.array(str)) -> np.array(np.int64):
  data = np.zeros_like(data_str)

  for i in range(data_str.size):
    data[i] = np.int64( data_str[i].replace('.', '') )
  return data

def conversione_anni(data_str: np.array(str)) -> np.array(np.int64):
  anni  = np.zeros_like(data_str)
  a0 = 1862
  for i in range(anni.size):
    anni[i] = a0
    a0 += 1
  return anni

anni        = conversione_anni(anno_str)
popolazione = conversione_stringhe_interi(popolazione_str)
nati        = conversione_stringhe_interi(nati_str)
morti       = conversione_stringhe_interi(morti_str)


# ===============================
# creazione di un nuovo dataframe
# ===============================

demo_df = pd.DataFrame(index = range(anni.size),  columns=['anno', 'nati', 'morti', "popolazione"]).fillna(0)
demo_df['anno'] = anni
demo_df['nati'] = nati
demo_df['morti'] = morti
demo_df['popolazione'] = popolazione


# ===========================================
# creazione di un nuovo file csv
# ===========================================
demo_df.to_csv('demo_italia_1862_2021.csv')
display(demo_df)



anno     nati   morti popolazione
0    1862   991000  815000    26328000
1    1863  1026000  824000    26507000
2    1864  1000000  802000    26712000
3    1865  1024000  812000    26915000
4    1866  1044000  798000    27131000
..    ...      ...     ...         ...
156  2018   439747  633133    59937769
157  2019   420084  634417    59816673
158  2020   404892  740317    59641488
159  2021   400249  701346    59236213
160  2022   392598  713499    59030133

[161 rows x 4 columns]

In [31]:
# andamento della popolazione
# se e come fare la media sull'andamento della popolazione

pop_seq = []

for popolazione in demo_df['popolazione']:
    pop_seq.append(popolazione)

anno_seq = []
for anno in demo_df['anno']:
  anno_seq.append(anno)

nati_seq = []
for morti in demo_df['nati']:
  nati_seq.append(nati)

morti_seq = []
for morti in demo_df['morti']:
  morti_seq.append(morti)

df_morti = morti_seq
df_nascite = nati_seq
df_anni = anno_seq
df_popolazione = pop_seq



